In [7]:
import sqlite3
import tkinter as tk
from tkinter import messagebox, ttk

# Database connection
def connect_db():
    return sqlite3.connect("restaurant_system.db")

# Fetch all restaurants
def fetch_restaurants():
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM Restaurants")
    restaurants = cursor.fetchall()
    conn.close()
    return restaurants

# Fetch menu items for a specific restaurant
def fetch_menu_items(restaurant_id):
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM MenuItems WHERE RestaurantID = ?", (restaurant_id,))
    menu_items = cursor.fetchall()
    conn.close()
    return menu_items

# Place a new order
def place_order(restaurant_id, menu_item_id, quantity):
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute(
        "INSERT INTO Orders (RestaurantID, MenuItemID, Quantity) VALUES (?, ?, ?)",
        (restaurant_id, menu_item_id, quantity),
    )
    conn.commit()
    conn.close()
    messagebox.showinfo("Order", "Order placed successfully!")

# Submit feedback
def submit_feedback(restaurant_id, customer_name, feedback_text):
    conn = connect_db()
    cursor = conn.cursor()
    cursor.execute(
        "INSERT INTO Feedback (RestaurantID, CustomerName, FeedbackText) VALUES (?, ?, ?)",
        (restaurant_id, customer_name, feedback_text),
    )
    conn.commit()
    conn.close()
    messagebox.showinfo("Feedback", "Feedback submitted successfully!")

# Main Application Class
class RestaurantApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Restaurant Management System")

        # Main frame
        main_frame = tk.Frame(self.root)
        main_frame.pack(pady=20, padx=20)

        # Restaurant selection
        tk.Label(main_frame, text="Select Restaurant:").grid(row=0, column=0, padx=10, pady=10)
        self.restaurant_combobox = ttk.Combobox(main_frame, state="readonly")
        self.restaurant_combobox.grid(row=0, column=1, padx=10, pady=10)
        self.restaurant_combobox.bind("<<ComboboxSelected>>", self.load_menu_items)

        self.load_restaurants()

        # Menu items list
        tk.Label(main_frame, text="Menu Items:").grid(row=1, column=0, padx=10, pady=10)
        self.menu_items_listbox = tk.Listbox(main_frame, width=40, height=10)
        self.menu_items_listbox.grid(row=1, column=1, padx=10, pady=10)

        # Order section
        tk.Label(main_frame, text="Quantity:").grid(row=2, column=0, padx=10, pady=10)
        self.quantity_entry = tk.Entry(main_frame)
        self.quantity_entry.grid(row=2, column=1, padx=10, pady=10)

        self.order_button = tk.Button(main_frame, text="Place Order", command=self.place_order)
        self.order_button.grid(row=3, column=1, pady=10)

        # Feedback section
        feedback_frame = tk.Frame(self.root)
        feedback_frame.pack(pady=20, padx=20)

        tk.Label(feedback_frame, text="Customer Name:").grid(row=0, column=0, padx=10, pady=10)
        self.customer_name_entry = tk.Entry(feedback_frame)
        self.customer_name_entry.grid(row=0, column=1, padx=10, pady=10)

        tk.Label(feedback_frame, text="Feedback:").grid(row=1, column=0, padx=10, pady=10)
        self.feedback_text_entry = tk.Entry(feedback_frame, width=40)
        self.feedback_text_entry.grid(row=1, column=1, padx=10, pady=10)

        self.feedback_button = tk.Button(feedback_frame, text="Submit Feedback", command=self.submit_feedback)
        self.feedback_button.grid(row=2, column=1, pady=10)

    def load_restaurants(self):
        restaurants = fetch_restaurants()
        self.restaurant_combobox["values"] = [f"{r[1]} (ID: {r[0]})" for r in restaurants]

    def load_menu_items(self, event):
        selection = self.restaurant_combobox.get()
        restaurant_id = int(selection.split("ID:")[-1].strip().strip(")"))
        menu_items = fetch_menu_items(restaurant_id)
        self.menu_items_listbox.delete(0, tk.END)
        for item in menu_items:
            self.menu_items_listbox.insert(tk.END, f"{item[2]} - ${item[3]} (ID: {item[0]})")

    def place_order(self):
        try:
            selection = self.restaurant_combobox.get()
            restaurant_id = int(selection.split("ID:")[-1].strip().strip(")"))
            menu_item = self.menu_items_listbox.get(tk.ACTIVE)
            menu_item_id = int(menu_item.split("ID:")[-1].strip().strip(")"))
            quantity = int(self.quantity_entry.get())
            place_order(restaurant_id, menu_item_id, quantity)
        except Exception as e:
            messagebox.showerror("Error", str(e))

    def submit_feedback(self):
        try:
            selection = self.restaurant_combobox.get()
            restaurant_id = int(selection.split("ID:")[-1].strip().strip(")"))
            customer_name = self.customer_name_entry.get()
            feedback_text = self.feedback_text_entry.get()
            submit_feedback(restaurant_id, customer_name, feedback_text)
        except Exception as e:
            messagebox.showerror("Error", str(e))

# Run application
if __name__ == "__main__":
    root = tk.Tk()
    app = RestaurantApp(root)
    root.mainloop()
